In [1]:
%load_ext lab_black

In [2]:
import math
import os
import random
import shutil
import json
import logging
import sys
from collections import defaultdict
from datetime import datetime
from io import TextIOWrapper
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from tqdm import tqdm

import utils
import networks

In [3]:
RESULT_DIR = Path("results/Effdet학습")

In [7]:
LR = 1e-4  # transfer learning이니깐 좀 작게 주는게 좋을 것 같아서 1e-4
BATCH_SIZE = 10
START_EPOCH = 1
SAM = False

In [8]:
n = datetime.now()
UID = f"{n.year:04d}{n.month:02d}{n.day:02d}-{n.hour:02d}{n.minute:02d}{n.second:02d}"
SEED = 20210309

In [9]:
utils.seed_everything(SEED, deterministic=False)
RESULT_DIR.mkdir(parents=True, exist_ok=True)
log = utils.CustomLogger(RESULT_DIR / f"log_{UID}.log", "a")
log.info("학습 시작")
log.info("UID:", UID)
log.info("SEED:", SEED)
log.info("LR:", LR)
log.info("BATCH_SIZE:", BATCH_SIZE)
log.info("START_EPOCH:", START_EPOCH)
log.info("SAM:", SAM)

[2021-03-13 21:09:45  INFO] 학습 시작
[2021-03-13 21:09:45  INFO] UID: 20210313-210942
[2021-03-13 21:09:45  INFO] SEED: 20210309
[2021-03-13 21:09:45  INFO] LR: 0.0001
[2021-03-13 21:09:45  INFO] BATCH_SIZE: 10
[2021-03-13 21:09:45  INFO] START_EPOCH: 1
[2021-03-13 21:09:45  INFO] SAM: False


---

## 데이터 로딩

In [10]:
train_imgs = np.array(sorted(list(Path("data/box2/train_imgs/").glob("*.jpg"))))
test_imgs = np.array(sorted(list(Path("data/box2/test_imgs/").glob("*.jpg"))))

In [11]:
df = pd.read_csv("data/ori/train_df.csv")
keypoints = df.to_numpy()[:, 1:].astype(np.float32)
keypoints = np.stack([keypoints[:, 0::2], keypoints[:, 1::2]], axis=2)

In [13]:
class ImageDataset(Dataset):
    def __init__(self, files, keypoints=None, padding=40):
        super().__init__()
        self.files = files
        self.keypoints = keypoints
        self.padding = padding

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        f = self.files[idx]
        img = imageio.imread(f)
        x = torch.as_tensor(img, dtype=torch.float32).permute(2, 0, 1) / 255.0

        if self.keypoints is not None:
            keypoints = self.keypoints[idx]
            xmin = keypoints[:, 0].min() - self.padding
            xmax = keypoints[:, 0].max() + self.padding
            ymin = keypoints[:, 1].min() - self.padding
            ymax = keypoints[:, 1].max() + self.padding
            target = {
                "labels": torch.tensor([1], dtype=torch.int64),
                "boxes": torch.tensor([[xmin, ymin, xmax, ymax]], dtype=torch.float32),
            }
            return x, target
        return x

In [ ]:
ds_train = ImageDataset(train_imgs, train_keypoints)
ds_valid = ImageDataset(valid_imgs, valid_keypoints)
ds_test = ImageDataset(test_imgs)
ds_total = utils.ChainDataset(ds_train, ds_valid)

In [3]:
model = networks.EfficientDet("efficientdet-d0", pretrained=True)